In [14]:
!pip uninstall -y tensorflow
!pip install transformers
!pip install tensorboard

     |################################| 3.8 MB 30.2 MB/s eta 0:00:01
     |################################| 1.3 MB 77.3 MB/s eta 0:00:01
     |################################| 88 kB 12.9 MB/s eta 0:00:01
     |################################| 104 kB 80.8 MB/s eta 0:00:01
     |################################| 582 kB 77.9 MB/s eta 0:00:01
     |################################| 2.7 MB 76.6 MB/s eta 0:00:01
     |################################| 58 kB 10.6 MB/s eta 0:00:01
     |################################| 76 kB 8.3 MB/s  eta 0:00:01
     |################################| 155 kB 73.3 MB/s eta 0:00:01
     |################################| 147 kB 74.1 MB/s eta 0:00:01
     |################################| 77 kB 9.8 MB/s  eta 0:00:01
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=117798 sha256=22a719911a61ff4f5d9db25aaef44eb15baa29a6bb71db4950bae5a4954a52dd
  Stored in directory: /home/ubuntu/.cache/pip/wheels/c3/af/84/3962a6af7b4ab336e951b7877dcfb

In [1]:
!cat train.txt | wc -l

5834244


In [1]:
!split -l1000000 train.txt --verbose

creating file 'xaa'
creating file 'xab'
creating file 'xac'
creating file 'xad'
creating file 'xae'
creating file 'xaf'


In [3]:
!mkdir td

In [4]:
!mv xaa td/xaa.txt | mv xab td/xbb.txt | mv xac td/xac.txt | mv xad td/xad.txt | mv xae td/xae.txt | mv xaf td/xaf.txt

In [1]:
%%time

from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("td/*.txt")]

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 30.9 ms


In [6]:
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [7]:
!mkdir codeBERT
tokenizer.save("codeBERT")

['codeBERT/vocab.json', 'codeBERT/merges.txt']

In [2]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "codeBERT/vocab.json",
    "codeBERT/merges.txt",
)


In [3]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [4]:
tokenizer.encode("def get url ( url ) : __indent__ return url . status __dedent__")

Encoding(num_tokens=19, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing, original_str, normalized_str])

In [5]:
tokenizer.encode("def get url ( url ) : __indent__ return url . status __dedent__").tokens

['<s>',
 'def',
 'Ġget',
 'Ġurl',
 'Ġ(',
 'Ġurl',
 'Ġ)',
 'Ġ:',
 'Ġ__',
 'indent',
 '__',
 'Ġreturn',
 'Ġurl',
 'Ġ.',
 'Ġstatus',
 'Ġ__',
 'dedent',
 '__',
 '</s>']

In [12]:
!nvidia-smi

Mon Mar  9 22:29:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   33C    P0    27W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [13]:
import torch
torch.cuda.is_available()

True

In [9]:
!wget -c https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py

--2020-03-09 11:59:52--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34666 (34K) [text/plain]
Saving to: 'run_language_modeling.py'

run_language_modeli 100%[===================>]  33.85K  --.-KB/s    in 0.001s  

2020-03-09 11:59:52 (29.4 MB/s) - 'run_language_modeling.py' saved [34666/34666]



In [14]:
import json
config = {
    "architectures": [
        "RobertaForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "layer_norm_eps": 1e-05,
    "max_position_embeddings": 512,
    "model_type": "roberta",
    "num_attention_heads": 12,
    "num_hidden_layers": 6,
    "type_vocab_size": 1,
    "vocab_size": 52000
}
with open("codeBERT/config.json", 'w') as fp:
    json.dump(config, fp)

tokenizer_config = {
    "max_len": 512
}
with open("codeBERT/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [6]:
!rm -f small_eval.txt small_train.txt

In [7]:
!head -1000000 train.txt > small_train.txt

In [8]:
!tail -10000 train.txt > small_eval.txt

In [9]:
train_file = "small_train.txt"

In [10]:
eval_file = "small_eval.txt"

In [11]:
!rm -rf codeBERT-small-v1

In [12]:
cmd ="""
  CUDA_LAUNCH_BLOCKING=1 python run_language_modeling.py
  --train_data_file {}
  --eval_data_file {}
  --output_dir ./codeBERT-small-v1
    --model_type roberta
    --mlm
    --config_name ./codeBERT
    --tokenizer_name ./codeBERT
    --do_train
    --do_eval
    --line_by_line
    --learning_rate 1e-4
    --num_train_epochs 1
    --save_total_limit 2
    --save_steps 2000
    --per_gpu_train_batch_size 4
    --seed 42
""".format(train_file, eval_file).replace("\n", " ")

In [13]:
cmd

'   CUDA_LAUNCH_BLOCKING=1 python run_language_modeling.py   --train_data_file small_train.txt   --eval_data_file small_eval.txt   --output_dir ./codeBERT-small-v1     --model_type roberta     --mlm     --config_name ./codeBERT     --tokenizer_name ./codeBERT     --do_train     --do_eval     --line_by_line     --learning_rate 1e-4     --num_train_epochs 1     --save_total_limit 2     --save_steps 2000     --per_gpu_train_batch_size 4     --seed 42 '

In [22]:
%%time
!{cmd}

03/09/2020 22:29:16 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
03/09/2020 22:29:16 - INFO - transformers.configuration_utils -   loading configuration file ./codeBERT/config.json
03/09/2020 22:29:16 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_h

Iteration:   5%|#5                          | 673/12500 [00:59<17:18, 11.39it/s]


Iteration:   8%|##2                        | 1061/12500 [01:33<16:25, 11.60it/s]


Iteration:  12%|###1                       | 1449/12500 [02:07<16:16, 11.31it/s]


Iteration:  15%|###9                       | 1837/12500 [02:41<15:37, 11.37it/s]


...

Iteration:  89%|#######################   | 11085/12500 [16:09<02:00, 11.70it/s]


Iteration:  92%|#######################8  | 11473/12500 [16:43<01:29, 11.48it/s]


Iteration:  95%|########################6 | 11861/12500 [17:16<00:55, 11.61it/s]


Iteration:  96%|########################9 | 11999/12500 [17:28<00:43, 11.59it/s]03/09/2020 22:46:59 - INFO - transformers.configuration_utils -   Configuration saved in ./codeBERT-small-v1/checkpoint-12000/config.json
03/09/2020 22:47:00 - INFO - transformers.modeling_utils -   Model weights saved in ./codeBERT-small-v1/checkpoint-12000/pytorch_model.bin
03/09/2020 22:47:00 - INFO - __main__ -   Saving model checkpoint to ./codeBERT-small-v1/checkpoint-12000
03/09/2020 22:47:00 - INFO - __main__ -   Deleting older checkpoint [./codeBERT-small-v1/checkpoint-8000] due to args.save_total_limit
03/09/2020 22:47:01 - INFO - __main__ -   Saving optimizer and scheduler states to ./codeBERT-small-v1/checkpoint-12000

Iteration:  98%|#########################4| 12233/12500 [17:50<00:22, 11.63it/s]


Epoch: 100%|####################################| 1/1 [18:12<00:00, 1092.92s/it]
03/09/2020 22:47:44 - INFO - __main__ -    global_step = 12500, average loss = 6.064623246898651
03/09/2020 22:47:44 - INFO - __main__ -   Saving model checkpoint to ./codeBERT-small-v1
03/09/2020 22:47:44 - INFO - transformers.configuration_utils -   Configuration saved in ./codeBERT-small-v1/config.json
03/09/2020 22:47:44 - INFO - transformers.modeling_utils -   Model weights saved in ./codeBERT-small-v1/pytorch_model.bin
03/09/2020 22:47:44 - INFO - transformers.configuration_utils -   loading configuration file ./codeBERT-small-v1/config.json
03/09/2020 22:47:44 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2